# TrimReads 演示笔记本

![Bioinformatics](https://images.unsplash.com/photo-1581591524425-c7e0978865fc?ixlib=rb-4.0.3&ixid=M3wxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8fA%3D%3D&auto=format&fit=crop&w=1740&q=80)

本笔记本演示了 TrimReads 包的功能，这是一个用于高通量测序数据质量修剪的 Python 工具。

## 1. 安装和导入

首先，我们安装 TrimReads 包并导入必要的库。

In [ ]:
# 安装 TrimReads 包
!pip install TrimReads

In [ ]:
# 导入必要的库
import os
import tempfile
import numpy as np
import matplotlib.pyplot as plt
from Bio.SeqIO.QualityIO import FastqGeneralIterator

# 导入 TrimReads 包
from trimreads import trimreads, fastq_parser

# 设置绘图风格
plt.style.use('ggplot')
%config InlineBackend.figure_format = 'retina'

## 2. 创建模拟数据

我们将创建一个模拟的 FASTQ 文件，其中包含具有特定质量模式的测序 reads。

In [ ]:
def create_simulated_fastq(file_path, num_reads=1000, read_length=150):
    """
    创建模拟的 FASTQ 文件
    
    参数:
        file_path: 输出文件路径
        num_reads: reads 数量
        read_length: 每个 read 的长度
    """
    bases = "ACGT"
    
    # 打开文件
    if file_path.endswith('.gz'):
        f = gzip.open(file_path, 'wt')
    else:
        f = open(file_path, 'w')
    
    with f:
        for i in range(num_reads):
            # 生成随机序列
            sequence = ''.join(np.random.choice(list(bases), size=read_length))
            
            # 生成质量分数:
            # - 中间高质量 (Q30-40)
            # - 两端低质量 (Q0-20)
            quality = []
            for pos in range(read_length):
                # 计算位置因子 (两端为0，中间为1)
                pos_factor = min(pos, read_length - pos - 1) / (read_length / 2)
                pos_factor = min(1.0, pos_factor)  # 上限为1.0
                
                # 基础质量: 中间高，两端低
                base_quality = int(20 + 20 * pos_factor)
                quality.append(trimreads.score_to_phred(base_quality))
            quality = ''.join(quality)
            
            # 写入 FASTQ 记录
            f.write(f"@SIMULATED_READ_{i}\n")
            f.write(f"{sequence}\n")
            f.write("+\n")
            f.write(f"{quality}\n")
    
    print(f"已创建模拟 FASTQ 文件: {file_path}")
    print(f"  Reads 数量: {num_reads}, 长度: {read_length} bp")

# 创建临时目录
temp_dir = tempfile.mkdtemp()
sim_fastq = os.path.join(temp_dir, "simulated.fastq")

# 生成模拟数据
create_simulated_fastq(sim_fastq, num_reads=5000, read_length=150)

## 3. 数据质量分析

在修剪前，我们先分析数据的质量特征。

In [ ]:
def plot_quality_profile(file_path, title=""):
    """
    绘制质量分布图
    """
    # 初始化质量矩阵
    quality_matrix = []
    
    # 读取 FASTQ 文件
    with fastq_parser.FastqParser(file_path) as parser:
        for record in parser.parse():
            # 将质量字符串转换为分数
            scores = [trimreads.phred_to_score(q) for q in record.quality]
            quality_matrix.append(scores)
    
    # 转换为 numpy 数组
    quality_matrix = np.array(quality_matrix)
    
    # 计算平均质量
    mean_quality = np.mean(quality_matrix, axis=0)
    
    # 绘制质量分布
    plt.figure(figsize=(12, 6))
    
    # 平均质量线
    plt.plot(mean_quality, color='dodgerblue', linewidth=2.5, label='平均质量')
    
    # 质量分位数
    plt.fill_between(
        range(len(mean_quality)),
        np.percentile(quality_matrix, 25, axis=0),
        np.percentile(quality_matrix, 75, axis=0),
        color='skyblue',
        alpha=0.3,
        label='25-75百分位'
    )
    
    # 添加阈值线
    plt.axhline(y=20, color='red', linestyle='--', alpha=0.7, label='Q20阈值')
    plt.axhline(y=30, color='green', linestyle='--', alpha=0.7, label='Q30阈值')
    
    # 设置图表属性
    plt.title(f"{title}质量分布图 (n={len(quality_matrix)} reads)", fontsize=14)
    plt.xlabel("碱基位置", fontsize=12)
    plt.ylabel("质量分数 (Phred)", fontsize=12)
    plt.ylim(0, 45)
    plt.grid(alpha=0.2)
    plt.legend()
    
    # 显示图表
    plt.tight_layout()
    plt.show()
    
    # 返回统计数据
    return {
        "mean_quality": mean_quality,
        "quality_matrix": quality_matrix
    }

# 分析原始数据质量
print("原始数据质量分析:")
orig_stats = plot_quality_profile(sim_fastq, "原始数据")

# 计算低质量碱基比例
low_quality_bases = np.sum(orig_stats["quality_matrix"] < 20)
total_bases = orig_stats["quality_matrix"].size
low_quality_percent = low_quality_bases / total_bases

print(f"低质量碱基比例 (Q<20): {low_quality_percent:.2%}")

## 4. 质量修剪

现在我们将使用三种不同的策略进行质量修剪：

1. **逐个碱基修剪**：从两端移除低质量碱基
2. **滑动窗口修剪**：基于窗口平均质量进行修剪
3. **组合方法**：结合前两种方法

In [ ]:
def run_trimming(input_file, output_file, method, threshold=25, window_size=10):
    """
    运行修剪流程
    
    参数:
        input_file: 输入 FASTQ 文件
        output_file: 输出 FASTQ 文件
        method: 修剪方法 ("base", "window", "combined")
        threshold: 质量阈值
        window_size: 窗口大小 (仅用于窗口方法)
    """
    print(f"正在运行 {method} 修剪...")
    
    if method == "base":
        stats = trimreads.process_fastq(
            input_file,
            output_file,
            base_threshold=threshold,
            min_length=50
        )
    elif method == "window":
        stats = trimreads.process_fastq(
            input_file,
            output_file,
            window_size=window_size,
            window_threshold=threshold,
            min_length=50
        )
    else:  # combined
        stats = trimreads.process_fastq(
            input_file,
            output_file,
            base_threshold=threshold,
            window_size=window_size,
            window_threshold=threshold,
            min_length=50
        )
    
    # 打印统计信息
    print(f"  Reads 总数: {stats['total_reads']}")
    print(f"  保留的 reads: {stats['passed_reads']} ({stats['passed_reads']/stats['total_reads']:.2%})")
    print(f"  丢弃的 reads: {stats['discarded_reads']} ({stats['discarded_reads']/stats['total_reads']:.2%})")
    
    return stats

# 创建输出文件路径
base_output = os.path.join(temp_dir, "base_trimmed.fastq")
window_output = os.path.join(temp_dir, "window_trimmed.fastq")
combined_output = os.path.join(temp_dir, "combined_trimmed.fastq")

# 运行各种修剪方法
print("\n=== 逐个碱基修剪 ===")
base_stats = run_trimming(sim_fastq, base_output, "base", threshold=25)

print("\n=== 滑动窗口修剪 ===")
window_stats = run_trimming(sim_fastq, window_output, "window", threshold=20, window_size=10)

print("\n=== 组合方法修剪 ===")
combined_stats = run_trimming(sim_fastq, combined_output, "combined", threshold=25, window_size=10)

## 5. 修剪结果分析

现在我们来比较不同修剪方法的效果。

In [ ]:
# 分析修剪后的数据质量
print("\n=== 修剪后质量分析 ===")

print("\n逐个碱基修剪结果:")
base_stats_qual = plot_quality_profile(base_output, "逐个碱基修剪")

print("\n滑动窗口修剪结果:")
window_stats_qual = plot_quality_profile(window_output, "滑动窗口修剪")

print("\n组合方法修剪结果:")
combined_stats_qual = plot_quality_profile(combined_output, "组合方法修剪")

In [ ]:
def compare_trimming_results(orig_stats, base_stats, window_stats, combined_stats):
    """
    比较不同修剪方法的结果
    """
    # 准备数据
    methods = ['原始数据', '逐个碱基', '滑动窗口', '组合方法']
    
    # 平均质量
    avg_qualities = [
        np.mean(orig_stats["quality_matrix"]),
        np.mean(base_stats_qual["quality_matrix"]),
        np.mean(window_stats_qual["quality_matrix"]),
        np.mean(combined_stats_qual["quality_matrix"])
    ]
    
    # 低质量碱基比例
    low_qual_percent = [
        np.sum(orig_stats["quality_matrix"] < 20) / orig_stats["quality_matrix"].size,
        np.sum(base_stats_qual["quality_matrix"] < 20) / base_stats_qual["quality_matrix"].size,
        np.sum(window_stats_qual["quality_matrix"] < 20) / window_stats_qual["quality_matrix"].size,
        np.sum(combined_stats_qual["quality_matrix"] < 20) / combined_stats_qual["quality_matrix"].size
    ]
    
    # 平均 read 长度
    avg_lengths = [
        orig_stats["quality_matrix"].shape[1],
        base_stats_qual["quality_matrix"].shape[1],
        window_stats_qual["quality_matrix"].shape[1],
        combined_stats_qual["quality_matrix"].shape[1]
    ]
    
    # 创建图表
    fig, ax = plt.subplots(3, 1, figsize=(10, 12))
    
    # 平均质量
    bars1 = ax[0].bar(methods, avg_qualities, color=['#4C72B0', '#55A868', '#C44E52', '#8172B2'])
    ax[0].set_title("平均质量分数比较", fontsize=14)
    ax[0].set_ylabel("平均质量 (Phred)")
    ax[0].grid(axis='y', alpha=0.3)
    
    # 添加数据标签
    for bar in bars1:
        height = bar.get_height()
        ax[0].annotate(f'{height:.1f}',
                      xy=(bar.get_x() + bar.get_width() / 2, height),
                      xytext=(0, 3),  # 3 points vertical offset
                      textcoords="offset points",
                      ha='center', va='bottom')
    
    # 低质量碱基比例
    bars2 = ax[1].bar(methods, low_qual_percent, color=['#4C72B0', '#55A868', '#C44E52', '#8172B2'])
    ax[1].set_title("低质量碱基比例 (Q<20) 比较", fontsize=14)
    ax[1].set_ylabel("比例")
    ax[1].grid(axis='y', alpha=0.3)
    
    # 添加数据标签
    for bar in bars2:
        height = bar.get_height()
        ax[1].annotate(f'{height:.1%}',
                      xy=(bar.get_x() + bar.get_width() / 2, height),
                      xytext=(0, 3),  # 3 points vertical offset
                      textcoords="offset points",
                      ha='center', va='bottom')
    
    # 平均 read 长度
    bars3 = ax[2].bar(methods, avg_lengths, color=['#4C72B0', '#55A868', '#C44E52', '#8172B2'])
    ax[2].set_title("平均 read 长度比较", fontsize=14)
    ax[2].set_ylabel("碱基数")
    ax[2].grid(axis='y', alpha=0.3)
    
    # 添加数据标签
    for bar in bars3:
        height = bar.get_height()
        ax[2].annotate(f'{height:.1f}',
                      xy=(bar.get_x() + bar.get_width() / 2, height),
                      xytext=(0, 3),  # 3 points vertical offset
                      textcoords="offset points",
                      ha='center', va='bottom')
    
    plt.tight_layout()
    plt.show()

# 比较结果
compare_trimming_results(orig_stats, base_stats_qual, window_stats_qual, combined_stats_qual)

## 6. 真实数据示例 (可选)

如果您有互联网连接，可以尝试使用真实的测序数据。

In [ ]:
def download_real_data(output_dir, accession="SRR15338479"):
    """
    下载真实测序数据
    """
    import subprocess
    
    url = f"https://sra-pub-run-odp.s3.amazonaws.com/sra/{accession}/{accession}"
    fastq_gz = os.path.join(output_dir, f"{accession}.fastq.gz")
    
    print(f"正在从ENA下载真实测序数据 ({accession})...")
    
    try:
        # 尝试使用fasterq-dump (需要安装SRA Toolkit)
        subprocess.run(["fasterq-dump", "--progress", accession], check=True, cwd=output_dir)
        # 压缩为gzip
        subprocess.run(["gzip", f"{accession}.fastq"], check=True, cwd=output_dir)
        print("下载完成")
        return fastq_gz
    except (FileNotFoundError, subprocess.CalledProcessError):
        print("fasterq-dump不可用，使用直接下载...")
        try:
            subprocess.run(["curl", "-o", fastq_gz, url], check=True)
            print("下载完成")
            return fastq_gz
        except subprocess.CalledProcessError:
            print("下载真实数据失败")
            return None

# 用户选择是否下载真实数据
use_real_data = False  # 更改为True以启用真实数据下载

if use_real_data:
    real_fastq = download_real_data(temp_dir)
    
    if real_fastq and os.path.exists(real_fastq):
        print("\n=== 真实数据质量分析 ===")
        real_stats = plot_quality_profile(real_fastq, "真实数据 (COVID-19测序)")
        
        # 修剪真实数据
        real_output = os.path.join(temp_dir, "real_trimmed.fastq.gz")
        print("\n运行真实数据修剪...")
        real_trim_stats = run_trimming(
            real_fastq, 
            real_output, 
            "combined", 
            threshold=25, 
            window_size=10
        )
        
        # 分析修剪后质量
        print("\n=== 修剪后质量分析 ===")
        real_trimmed_stats = plot_quality_profile(real_output, "修剪后真实数据")
    else:
        print("无法获取真实数据，跳过此部分")
else:
    print("跳过真实数据部分，如需启用请设置use_real_data=True")

## 7. 结论

通过本演示，我们展示了 TrimReads 包的主要功能：

1. **逐个碱基修剪**：有效移除两端的低质量碱基
2. **滑动窗口修剪**：基于局部平均质量进行修剪
3. **组合方法**：提供最全面的质量提升

### 关键发现：
- 所有修剪方法都显著提高了平均质量分数
- 组合方法在质量提升方面表现最佳，但保留的序列长度最短
- 滑动窗口修剪在保留长度和质量之间提供了良好的平衡

### 下一步：
1. 在您自己的数据上尝试 TrimReads
2. 调整参数以获得最佳结果
3. 探索高级功能，如并行处理和自定义质量编码

In [ ]:
# 清理临时文件
import shutil
shutil.rmtree(temp_dir)
print(f"已清理临时文件: {temp_dir}")

## 参考文献和资源

1. [TrimReads GitHub 仓库](https://github.com/yourusername/TrimReads)
2. [Biopython 文档](https://biopython.org)
3. [European Nucleotide Archive](https://www.ebi.ac.uk/ena)
4. Andrews S. (2010). FastQC: A Quality Control Tool for High Throughput Sequence Data.